## Install dependencies

In [1]:
!pip install torchaudio omegaconf soundfile

  Using cached SoundFile-0.10.3.post1-py2.py3.cp26.cp27.cp32.cp33.cp34.cp35.cp36.pp27.pp32.pp33-none-win_amd64.whl (689 kB)


## Import Dependencies

In [2]:
import torch
import zipfile
import torchaudio
from glob import glob

In [3]:
device = torch.device('cpu')  # gpu also works, but our models are fast enough for CPU

model, decoder, utils = torch.hub.load(repo_or_dir='snakers4/silero-models',model='silero_stt',language='en', # also available 'de', 'es'
                                       device=device)

Using cache found in C:\Users\Sukaran/.cache\torch\hub\snakers4_silero-models_master
100.0%


In [4]:
(read_batch, split_into_batches,
 read_audio, prepare_model_input) = utils  # see function signature for details

## sample file

In [5]:
# download a single file, any format compatible with TorchAudio (soundfile backend)
torch.hub.download_url_to_file('https://opus-codec.org/static/examples/samples/speech_orig.wav',
                               dst ='speech_orig.wav', progress=True)

100.0%


In [22]:
import os
if 'output.wav' in os.listdir():
    os.remove('output.wav')

## taking audio as input

In [23]:
import pyaudio
import wave

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "output.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

* recording
* done recording


In [24]:
test_files = glob('output.wav')
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]),
                            device=device)

In [25]:
output = model(input)
for example in output:
    print(decoder(example.cpu()))

such inintdocurvwikiedia
